In [74]:
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array , load_img
from tensorflow.keras.layers import Conv2D , Flatten , MaxPool2D , Dense , GlobalAveragePooling2D , AveragePooling2D , Dropout , Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os
import tqdm

In [75]:
directory = '/content/drive/My Drive/csv files/Fire-Detection'
categories = ['fire' , 'no_fire']

In [76]:
lr = 1e-4
epochs = 20
batch_size = 32

In [77]:
data = []
labels = []

for category in categories:
  path = os.path.join(directory , category)
  for img in os.listdir(path):
    image_path = os.path.join( path , img)
    image = load_img(image_path , target_size=(224,224))
    image = img_to_array(image)
    data.append(image)
    labels.append(category)

In [78]:
#save data and label in .npy file
np.savez_compressed('/content/drive/My Drive/Colab Notebooks/Tensorflow 2.x/fire_detection/fire_dataset.npz' , data , labels)

In [79]:
# load the dataset.npz file
data = np.load('/content/drive/My Drive/Colab Notebooks/Tensorflow 2.x/fire_detection/fire_dataset.npz')

In [80]:
data , label = data['arr_0'] , data['arr_1']
# print( data.shape)
# print( label.shape)

In [82]:
lb = LabelBinarizer()
labels = lb.fit_transform(label)
labels = to_categorical(labels)

In [83]:
# convert the data into numpy array
data = np.array(data , dtype='float32')
labels = np.array(labels)

In [85]:
X_train , X_test , y_train , y_test = train_test_split(data,labels ,test_size=0.2, random_state = 101 ,)

In [86]:
datagen = ImageDataGenerator(rotation_range=20,
                              zoom_range=0.15,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.15,
                              horizontal_flip=True,
                              fill_mode="nearest",
                            )

In [87]:
base_model = ResNet50(input_shape=(224,224 , 3) , include_top=False , weights='imagenet')

In [88]:
head_model = base_model.output
head_model= AveragePooling2D(7)(head_model)
head_model = Flatten()(head_model)
head_model = Dense( 128 , activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense( 2 , activation='softmax')(head_model)

In [89]:
model = Model(inputs = base_model.input , outputs = head_model)

for layer in base_model.layers:
  layer.trainable = False

In [90]:
optim = optimizers.Adam(learning_rate=lr , decay = lr/batch_size)
model.compile(optim , loss=keras.losses.categorical_crossentropy , metrics=['accuracy'])

In [91]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [92]:
model.fit(datagen.flow(X_train , y_train, batch_size = batch_size) , 
                  steps_per_epoch=len(X_train)//batch_size , 
                  validation_data=(X_test , y_test) , 
                  validation_steps= len(X_test)//batch_size , 
                  epochs = epochs)

Epoch 1/20
16/16 [==============================] - 6s 368ms/step - loss: 0.7258 - accuracy: 0.7848 - val_loss: 0.3978 - val_accuracy: 0.8244
Epoch 2/20
16/16 [==============================] - 5s 302ms/step - loss: 0.4707 - accuracy: 0.8238 - val_loss: 0.3326 - val_accuracy: 0.8855
Epoch 3/20
16/16 [==============================] - 5s 302ms/step - loss: 0.3467 - accuracy: 0.8873 - val_loss: 0.3216 - val_accuracy: 0.8855
Epoch 4/20
16/16 [==============================] - 5s 315ms/step - loss: 0.2809 - accuracy: 0.8984 - val_loss: 0.3077 - val_accuracy: 0.8855
Epoch 5/20
16/16 [==============================] - 5s 314ms/step - loss: 0.2460 - accuracy: 0.9062 - val_loss: 0.3015 - val_accuracy: 0.8855
Epoch 6/20
16/16 [==============================] - 5s 300ms/step - loss: 0.2030 - accuracy: 0.9201 - val_loss: 0.3394 - val_accuracy: 0.8931
Epoch 7/20
16/16 [==============================] - 5s 301ms/step - loss: 0.2112 - accuracy: 0.9098 - val_loss: 0.2934 - val_accuracy: 0.9084
Epoch 

In [93]:
model_json = model.to_json()
with open("/content/drive/My Drive/Colab Notebooks/Tensorflow 2.x/fire_detection/fire_detection.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Colab Notebooks/Tensorflow 2.x/fire_detection/fire_detection.h5")
print("Saved model to disk")

Saved model to disk
